# Sunbird RC API - Water Facility Demo (Public URL)

This notebook demonstrates how to interact with Sunbird RC Registry API for Water Facilities
deployed at https://sunbird-rc.akvotest.org.

## Prerequisites
- Sunbird RC services deployed at https://sunbird-rc.akvotest.org
- WaterFacility schema configured
- Keycloak `admin-api` client secret (from k8s secret `sunbird-rc`)

## Setup

In [ ]:
import requests
import json
import os
import pandas as pd
from getpass import getpass
from datetime import datetime
from IPython.display import display, Markdown, JSON

# Configuration
DOMAIN = "https://sunbird-rc.akvotest.org"
BASE_URL = f"{DOMAIN}/api/v1"
KEYCLOAK_URL = f"{DOMAIN}/auth/realms/sunbird-rc/protocol/openid-connect/token"

# Auth credentials (demo-api client)
CLIENT_ID = "demo-api"
CLIENT_SECRET = os.environ.get("DEMO_API_CLIENT_SECRET") or getpass("Enter demo-api client secret: ")

print(f"Registry API: {BASE_URL}")
print(f"Keycloak Token URL: {KEYCLOAK_URL}")

## 0. Authenticate with Keycloak

Obtain an access token using the `admin-api` client credentials.

In [ ]:
token_response = requests.post(KEYCLOAK_URL, data={
    "client_id": CLIENT_ID,
    "client_secret": CLIENT_SECRET,
    "grant_type": "client_credentials"
})

if token_response.status_code == 200:
    ACCESS_TOKEN = token_response.json()["access_token"]
    AUTH_HEADERS = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {ACCESS_TOKEN}"
    }
    print(f"Token obtained (expires in {token_response.json()['expires_in']}s)")
else:
    print(f"Auth failed: {token_response.status_code}")
    print(token_response.text)

## 1. Check Registry Health

In [ ]:
response = requests.get(f"{DOMAIN}/health")
health = response.json()

print(f"Status: {health['result']['healthy']}")
print(f"Service: {health['result']['name']}")
display(JSON(health, expanded=True))

## 2. List All Water Facilities

In [ ]:
response = requests.get(f"{BASE_URL}/WaterFacility", headers=AUTH_HEADERS)
data = response.json()

print(f"Total Water Facilities: {data['totalCount']}\n")

if data['totalCount'] > 0:
    df = pd.DataFrame(data['data'])
    desired_columns = ['wfName', 'wfId', 'typeOfWaterFacility', 'institution', 'foundingDate', 'waterCapacity', 'osid']
    columns = [col for col in desired_columns if col in df.columns]
    display(df[columns])

    if 'location' in df.columns:
        print("\nLocation Details:")
        location_df = pd.json_normalize(df['location'])
        location_df['wfName'] = df['wfName'].values
        display(location_df[['wfName', 'province', 'district', 'village']])
else:
    print("No water facilities found")

## 3. Create a New Water Facility

In [ ]:
new_facility = {
    "wfName": "IPAM Dago Pakar",
    "typeOfWaterFacility": "Water Treatment Plant",
    "location": {
        "province": "Jawa Barat",
        "district": "Kota Bandung",
        "village": "Dago"
    },
    "address": "Jl. Dago Pakar No. 12, Bandung 40135",
    "institution": "PDAM Tirtawening Kota Bandung",
    "foundingDate": "2018-06-15",
    "waterCapacity": 50000
}

response = requests.post(
    f"{BASE_URL}/WaterFacility",
    headers=AUTH_HEADERS,
    json=new_facility
)

if response.status_code == 200:
    result = response.json()
    osid = result['result']['WaterFacility']['osid']
    print("Water Facility created successfully!")
    print(f"osid: {osid}")

    fetch_response = requests.get(f"{BASE_URL}/WaterFacility/{osid}", headers=AUTH_HEADERS)
    if fetch_response.status_code == 200:
        created_facility = fetch_response.json()
        print(f"wfId: {created_facility.get('wfId', 'N/A')}")
        print(f"\nGenerated wfId format: WF-<PROVINCE>-<DISTRICT>-<TYPE>-<HASH>")
        display(JSON(created_facility, expanded=True))
else:
    print(f"Error: {response.status_code}")
    print(response.text)

## 4. Get a Specific Water Facility by ID

In [ ]:
response = requests.get(f"{BASE_URL}/WaterFacility", headers=AUTH_HEADERS)
facilities = response.json()['data']

if facilities:
    facility_id = facilities[0]['osid']
    print(f"Fetching water facility: {facility_id}\n")
    response = requests.get(f"{BASE_URL}/WaterFacility/{facility_id}", headers=AUTH_HEADERS)
    facility = response.json()
    display(JSON(facility, expanded=True))
else:
    print("No facilities found")

## 5. Update a Water Facility

In [ ]:
response = requests.get(f"{BASE_URL}/WaterFacility", headers=AUTH_HEADERS)
facilities = response.json()['data']

if facilities:
    facility_id = facilities[0]['osid']

    update_data = facilities[0].copy()
    update_data['waterCapacity'] = 75000
    update_data['address'] = '456 New River Road, Metro City, MC 12345'

    response = requests.put(
        f"{BASE_URL}/WaterFacility/{facility_id}",
        headers=AUTH_HEADERS,
        json=update_data
    )

    if response.status_code == 200:
        print("Water Facility updated successfully!")
        display(JSON(response.json(), expanded=True))
    else:
        print(f"Error: {response.status_code}")
        print(response.text)
else:
    print("No facilities available to update")

## 6. Search Water Facilities

In [ ]:
search_query = {
    "filters": {
        "typeOfWaterFacility": {"eq": "Water Treatment Plant"}
    }
}

response = requests.post(
    f"{BASE_URL}/WaterFacility/search",
    headers=AUTH_HEADERS,
    json=search_query
)

if response.status_code == 200:
    results = response.json()
    print(f"Found {results['totalCount']} Water Treatment Plants\n")

    if results['totalCount'] > 0:
        df = pd.DataFrame(results['data'])
        desired_columns = ['wfId', 'wfName', 'typeOfWaterFacility', 'institution', 'waterCapacity']
        columns = [col for col in desired_columns if col in df.columns]
        display(df[columns])

        if 'location' in df.columns:
            print("\nLocation Details:")
            location_df = pd.json_normalize(df['location'])
            location_df['wfName'] = df['wfName'].values
            if 'wfId' in df.columns:
                location_df['wfId'] = df['wfId'].values
                display(location_df[['wfId', 'wfName', 'province', 'district', 'village']])
            else:
                display(location_df[['wfName', 'province', 'district', 'village']])
    else:
        print("No results found")
else:
    print(f"Error: {response.status_code}")
    print(response.text)

## 7. Bulk Create Water Facilities

In [ ]:
# Sample bulk data with Indonesian locations
bulk_facilities = [
    {
        "wfName": "Waduk Jatiluhur",
        "typeOfWaterFacility": "Reservoir",
        "location": {
            "province": "Jawa Barat",
            "district": "Kabupaten Purwakarta",
            "village": "Jatiluhur"
        },
        "address": "Jl. Waduk Jatiluhur, Purwakarta 41152",
        "institution": "Perum Jasa Tirta II",
        "foundingDate": "1967-08-26",
        "waterCapacity": 1000000
    },
    {
        "wfName": "Pompa Distribusi Cikokol",
        "typeOfWaterFacility": "Pumping Station",
        "location": {
            "province": "Banten",
            "district": "Kota Tangerang",
            "village": "Cikokol"
        },
        "address": "Jl. MH Thamrin No. 45, Tangerang 15117",
        "institution": "PDAM Tirta Benteng Kota Tangerang",
        "foundingDate": "2015-03-10",
        "waterCapacity": 25000
    },
    {
        "wfName": "IPA Buaran",
        "typeOfWaterFacility": "Water Treatment Plant",
        "location": {
            "province": "DKI Jakarta",
            "district": "Jakarta Timur",
            "village": "Klender"
        },
        "address": "Jl. Raya Bekasi KM 18, Jakarta Timur 13930",
        "institution": "PAM Jaya",
        "foundingDate": "1990-05-20",
        "waterCapacity": 450000
    },
    {
        "wfName": "SWRO Nusa Penida",
        "typeOfWaterFacility": "Desalination Plant",
        "location": {
            "province": "Bali",
            "district": "Kabupaten Klungkung",
            "village": "Ped"
        },
        "address": "Desa Ped, Nusa Penida, Klungkung 80771",
        "institution": "PDAM Tirta Tohlangkir",
        "foundingDate": "2021-11-15",
        "waterCapacity": 5000
    },
    {
        "wfName": "Reservoir Cilandak",
        "typeOfWaterFacility": "Distribution Center",
        "location": {
            "province": "DKI Jakarta",
            "district": "Jakarta Selatan",
            "village": "Cilandak Barat"
        },
        "address": "Jl. Cilandak KKO, Jakarta Selatan 12560",
        "institution": "PAM Jaya",
        "foundingDate": "2010-09-01",
        "waterCapacity": 80000
    }
]

print(f"Creating {len(bulk_facilities)} water facilities...\n")

results = []
for i, facility in enumerate(bulk_facilities, 1):
    response = requests.post(
        f"{BASE_URL}/WaterFacility",
        headers=AUTH_HEADERS,
        json=facility
    )

    if response.status_code == 200:
        result = response.json()
        osid = result['result']['WaterFacility']['osid']

        fetch_response = requests.get(f"{BASE_URL}/WaterFacility/{osid}", headers=AUTH_HEADERS)
        wfId = "N/A"
        if fetch_response.status_code == 200:
            created = fetch_response.json()
            wfId = created.get('wfId', 'N/A')

        print(f"{i}. {facility['wfName']}")
        print(f"   wfId: {wfId}")
        print(f"   osid: {osid}\n")
        results.append({'status': 'success', 'osid': osid, 'wfId': wfId, 'name': facility['wfName']})
    else:
        print(f"{i}. {facility['wfName']} - FAILED ({response.status_code})")
        print(f"   Error: {response.text}\n")
        results.append({'status': 'failed', 'name': facility['wfName'], 'error': response.text})

print(f"{'='*50}")
print(f"Created {len([r for r in results if r['status'] == 'success'])} facilities")
print(f"Failed {len([r for r in results if r['status'] == 'failed'])} facilities")

## 7.1 Test Duplicate Rejection (wfId uniqueness)

The wfId is generated based on a hash of facility attributes (wfName, typeOfWaterFacility, location).
Attempting to create a facility with the same attributes should be rejected due to the unique constraint on wfId.

In [ ]:
duplicate_facility = {
    "wfName": "Waduk Jatiluhur",
    "typeOfWaterFacility": "Reservoir",
    "location": {
        "province": "Jawa Barat",
        "district": "Kabupaten Purwakarta",
        "village": "Jatiluhur"
    },
    "address": "Different address - should not matter",
    "institution": "Different Institution - should not matter",
    "foundingDate": "2024-01-01",
    "waterCapacity": 999999
}

print("Testing duplicate rejection...")
print(f"Attempting to create: {duplicate_facility['wfName']}")
print(f"Location: {duplicate_facility['location']['province']}, {duplicate_facility['location']['district']}")
print()

response = requests.post(
    f"{BASE_URL}/WaterFacility",
    headers=AUTH_HEADERS,
    json=duplicate_facility
)

if response.status_code == 200:
    result = response.json()
    osid = result['result']['WaterFacility']['osid']
    print(f"Unexpected: Facility was created with osid: {osid}")

    fetch_response = requests.get(f"{BASE_URL}/WaterFacility/{osid}", headers=AUTH_HEADERS)
    if fetch_response.status_code == 200:
        created = fetch_response.json()
        print(f"wfId: {created.get('wfId', 'N/A')}")
else:
    print(f"Expected behavior: Duplicate rejected!")
    print(f"Status code: {response.status_code}")
    try:
        error_detail = response.json()
        print(f"Error: {json.dumps(error_detail, indent=2)}")
    except:
        print(f"Error: {response.text}")

## 8. Export to CSV

In [ ]:
response = requests.get(f"{BASE_URL}/WaterFacility", headers=AUTH_HEADERS)
data = response.json()

if data['totalCount'] > 0:
    df = pd.DataFrame(data['data'])

    if 'location' in df.columns:
        location_df = pd.json_normalize(df['location'])
        location_df.columns = [f'location_{col}' for col in location_df.columns]
        df = pd.concat([df.drop('location', axis=1), location_df], axis=1)

    export_columns = [
        'wfName', 'wfId', 'typeOfWaterFacility', 'institution',
        'location_province', 'location_district', 'location_village',
        'address', 'foundingDate', 'waterCapacity',
        'osid', 'osCreatedAt'
    ]
    available_columns = [col for col in export_columns if col in df.columns]

    filename = f'water_facilities.csv'
    df[available_columns].to_csv(filename, index=False)

    print(f"Exported {len(df)} water facilities to: {filename}")
    display(df[available_columns].head())
else:
    print("No water facilities to export")

## 9. Delete All Water Facilities

In [ ]:
response = requests.get(f"{BASE_URL}/WaterFacility", headers=AUTH_HEADERS)
facilities = response.json()['data']

if facilities:
    print(f"Deleting {len(facilities)} water facilities...\n")

    deleted = 0
    failed = 0
    for facility in facilities:
        osid = facility['osid']
        name = facility['wfName']

        response = requests.delete(f"{BASE_URL}/WaterFacility/{osid}", headers=AUTH_HEADERS)

        if response.status_code == 200:
            print(f"Deleted: {name} ({osid})")
            deleted += 1
        else:
            print(f"Failed to delete: {name} ({osid})")
            failed += 1

    print(f"\nDeleted {deleted} facilities")
    if failed > 0:
        print(f"Failed {failed} facilities")
else:
    print("No water facilities to delete")

## Summary

This notebook demonstrated:
- Checking registry health
- Listing all water facilities
- Creating new water facilities with auto-generated wfId
- Reading specific facilities
- Updating facilities
- Searching facilities by type
- Bulk operations with wfId display
- Testing duplicate rejection (wfId uniqueness)
- Exporting to CSV
- Deleting all facilities

### wfId Generation

The `wfId` is automatically generated when creating a WaterFacility using the format:
```
WF-<PROVINCE_ABBR>-<DISTRICT_ABBR>-<TYPE_CODE>-<HASH>
```

**Example:** `WF-JAW-KAB-RES-CB37DF`

**Components:**
- `WF-` - Fixed prefix for Water Facility
- `PROVINCE_ABBR` - First 3 alphanumeric characters of province (uppercase)
- `DISTRICT_ABBR` - First 3 alphanumeric characters of district (uppercase)
- `TYPE_CODE` - Facility type code:
  - `WTP` - Water Treatment Plant
  - `RES` - Reservoir
  - `PS` - Pumping Station
  - `DC` - Distribution Center
  - `DES` - Desalination Plant
- `HASH` - 6-character SHA-256 hash of normalized attributes

**Hash inputs (normalized to lowercase, trimmed):**
- wfName
- typeOfWaterFacility
- location.province
- location.district
- location.village

**Uniqueness:** The wfId is marked as a unique index field. Duplicate facilities (same core attributes) will be rejected.

### Water Facility Schema Fields
**Required Fields:**
- `wfName` - Name of the facility
- `typeOfWaterFacility` - Type (enum)
- `location` - Administrative location object
  - `province` - Province name
  - `district` - District/Regency name
  - `village` - Village/Sub-district name
- `institution` - Responsible institution
- `foundingDate` - Date of establishment (YYYY-MM-DD)

**Auto-generated Fields:**
- `wfId` - System-generated unique ID (format: WF-XXX-XXX-TYPE-HASH)

**Optional Fields:**
- `address` - Free-text address
- `waterCapacity` - Capacity in cubic meters (0-1,000,000)

### Water Facility Types Supported
- Water Treatment Plant (WTP)
- Reservoir (RES)
- Pumping Station (PS)
- Distribution Center (DC)
- Desalination Plant (DES)

For more information, see the WaterFacility schema at `java/registry/src/main/resources/public/_schemas/WaterFacility.json`